<a href="https://colab.research.google.com/github/swafey-karanja/Model-training/blob/main/FourthYearProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VET-ASSISTANT LLM**

**Installing dependencies**

In [ ]:
!pip install gradientai --upgrade
!pip install langchain
!pip install -U gradient_haystack
!pip install regex
!pip install rouge
!pip install nltk
!pip install wandb
!pip install datasets
!pip install rouge
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.3
    Uninstalling pydantic-2.6.3:
      Successfully uninstalled pydantic-2.6.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## **1.Loading the Dataset**

In [ ]:
import json
import requests

# URL of the online repository where the dataset is hosted
dataset_url = "https://raw.githubusercontent.com/swafey-karanja/Model-training/main/data.json"

# Make an HTTP GET request to fetch the dataset
response = requests.get(dataset_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load the dataset from the response content
    train_dataset = json.loads(response.text)

    # Print the size of the dataset
    print("Dataset Size:", len(train_dataset))
else:
    # Print an error message if the request failed
    print("Failed to fetch dataset. Status code:", response.status_code)


Dataset Size: 1008


**Break the data into batches**

In [ ]:
def divide_into_Batches(number, chunk_size):  # Define a function to divide a number into chunks of a given size
    Batches = []
    while number > 0:
        if number >= chunk_size:
            Batches.append(chunk_size)
            number -= chunk_size
        else:
            Batches.append(number)
            break

    return Batches

Batches = divide_into_Batches(len(train_dataset), 100)  # Divide the dataset into chunks of 100 samples each
print("Batches size")
print(Batches)

Batches size
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 8]


## **2.Load the Base Model.**

Loading the Nous-Hermes-Llama2-13b base model

In [ ]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "t4YxBtnw9hQ4qVB8ia9P4K8XkPLoDoZZ"
os.environ['GRADIENT_WORKSPACE_ID'] = "c571b959-4ce8-474a-b3f3-398c7b347c57_workspace"

In [ ]:
import json
import os
from gradientai import Gradient
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate


os.environ['GRADIENT_ACCESS_TOKEN'] = "t4YxBtnw9hQ4qVB8ia9P4K8XkPLoDoZZ"
os.environ['GRADIENT_WORKSPACE_ID'] = "c571b959-4ce8-474a-b3f3-398c7b347c57_workspace"

gradient =  Gradient()

print("Available Base Models")
for i in gradient.list_models(only_base=True):
    print("\t",i)

base_model_id = "NousResearch/Nous-Hermes-Llama2-13b"
base_model_name = "nous-hermes2"
base_model = gradient.get_base_model(base_model_slug="nous-hermes2") # base model Nous-Hermes-Llama2-13b

print("\nBase Model Chosen :", base_model)

Available Base Models

Base Model Chosen : <gradientai._base_model.BaseModel object at 0x78da99a22170>


##**3. Creating a Model Adapter**

* Adapters are small, lightweight modules inserted between
existing layers of a pre-trained LLM. They act like "add-ons" that focus on learning task-specific information without modifying the core knowledge captured in the original model.

* The addapter server as the object that we are going to fine tune

In [ ]:
our_finetune_model_name="Llama2-13b/Vet-assistant"
Fine_Tune__adapter = base_model.create_model_adapter(
        name=our_finetune_model_name, # base mode nous hermis
        learning_rate=0.00005, #  Determines how fast a model updates its knowledge during fine-tuning.
        rank=8,  # Dimensionality

    )

# default hyperparameters Frozen
hyperparameters = {
                  "block_size": 1024,
                  "model_max_length": 2048,
                  "padding": "right",
                  "use_flash_attention_2": False,
                  "disable_gradient_checkpointing": False,
                  "logging_steps": -1,
                  "evaluation_strategy": "epoch",
                  "save_total_limit": 1,
                  "save_strategy": "epoch",
                  "auto_find_batch_size": False,
                  "mixed_precision": "fp16",
                  "epochs": 3,
                  "batch_size": 100,
                  "warmup_ratio": 0.1,
                  "gradient_accumulation": 1,
                  "optimizer": "adamw_torch",
                  "scheduler": "linear",
                  "weight_decay": 0,
                  "max_grad_norm": 1,
                  "seed": 42,
                  "apply_chat_template": False,
                  "quantization": "int4",
                  "target_modules": "",
                  "merge_adapter": False,
                  "peft": True,
                  "lora_r": 16,
                  "lora_alpha": 32,
                  "lora_dropout": 0.05
  }


print(f"Base model id                : {Fine_Tune__adapter._base_model_id}")
print(f"Fine tune model Name         : { Fine_Tune__adapter.name}")
print(f"Fine tune model adapter id   : {Fine_Tune__adapter.id}")

print("\n\n")
print("Size of object in memory, in bytes.", Fine_Tune__adapter.__format__.__sizeof__()) # Size of object in memory, in bytes.
Fine_Tune__adapter.__dict__

Base model id                : cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model
Fine tune model Name         : Llama2-13b/Vet-assistant
Fine tune model adapter id   : 640e55a6-3046-4824-8831-10e7ae54a1d6_model_adapter



Size of object in memory, in bytes. 56


{'_api_instance': <gradientai.openapi.client.api.models_api.ModelsApi at 0x78da99a21630>,
 '_id': '640e55a6-3046-4824-8831-10e7ae54a1d6_model_adapter',
 '_workspace_id': 'c571b959-4ce8-474a-b3f3-398c7b347c57_workspace',
 '_async_semaphore': <asyncio.locks.Semaphore object at 0x78da99a22bc0 [unlocked, value:8]>,
 '_base_model_id': 'cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model',
 '_name': 'Llama2-13b/Vet-assistant'}

## **4. Fine Tuning The Adaptor**

For this case we will be performing Laura-based finetuning which means that we are freezing about 99% of the layers and then finetuning an adapter on top of it.

LoRA: Low-Rank Adaptation of Large Language Models is a novel technique introduced by Microsoft researchers to deal with the problem of fine-tuning large-language models with billions of parameters

 LoRA proposes to freeze pre-trained model weights and inject trainable layers (rank-decomposition matrices) in each transformer block. This greatly reduces the number of trainable parameters and GPU memory requirements since gradients don't need to be computed for most model weights


why LoRA finetuning:

1. Faster Training: Since only the added task-specific layers are trained while the pre-trained model's parameters remain frozen, the fine-tuning process is generally faster compared to training a model from scratch
2. Computation requirements are lower. We could create a full fine-tuned model in a 2080 Ti with 11 GB of VRAM!
3. Trained weights are  much smaller. Because the original model is frozen and we inject new layers to be trained

 [for more info](https://huggingface.co/blog/lora)

In [ ]:
print(f"Our Model id:  {Fine_Tune__adapter.id}")
num_epochs = 2  # num_epochs is the number of times you fine-tune the model # more epochs tends to get better results, but you also run the risk of "overfitting"
count = 0
print("================================================================\n")
print("Fine tuning . . .\n")
while count < num_epochs:
    print(f"Fine-tuning the model, iteration {count + 1}")
    s = 0
    n = 1
    for Batch in Batches:
        print(f"Batch {n} range: {s} : {(s + Batch)}")

        # Try to fine-tune the model with the chunk of samples,
        while True:
            try:
                metric = Fine_Tune__adapter.fine_tune(samples=train_dataset[s: s + Batch])
                print(f"\t Batch {n} Evaluation :", metric)
                break
            except:
                pass



        s += Batch
        n += 1
    count = count + 1

Our Model id:  640e55a6-3046-4824-8831-10e7ae54a1d6_model_adapter

Fine tuning . . .

Fine-tuning the model, iteration 1
Batch 1 range: 0 : 100
	 Batch 1 Evaluation : number_of_trainable_tokens=8007 sum_loss=18044.285
Batch 2 range: 100 : 200
	 Batch 2 Evaluation : number_of_trainable_tokens=8043 sum_loss=11166.195
Batch 3 range: 200 : 300
	 Batch 3 Evaluation : number_of_trainable_tokens=9278 sum_loss=10463.905
Batch 4 range: 300 : 400
	 Batch 4 Evaluation : number_of_trainable_tokens=10454 sum_loss=11904.527
Batch 5 range: 400 : 500
	 Batch 5 Evaluation : number_of_trainable_tokens=13087 sum_loss=12875.653
Batch 6 range: 500 : 600
	 Batch 6 Evaluation : number_of_trainable_tokens=16332 sum_loss=17145.766
Batch 7 range: 600 : 700
	 Batch 7 Evaluation : number_of_trainable_tokens=18421 sum_loss=17443.84
Batch 8 range: 700 : 800
	 Batch 8 Evaluation : number_of_trainable_tokens=18931 sum_loss=17171.729
Batch 9 range: 800 : 900
	 Batch 9 Evaluation : number_of_trainable_tokens=13457 sum_

## **5.Model Inference**

"model inference" typically refers to the process of using a trained model to make predictions on new, unseen data. Fine-tuning a model involves taking a pre-trained model and further training it on a specific task or dataset to improve its performance.

When fine-tuning a model, the process of inference remains the same as with any other trained model. Once the fine-tuning is complete, you can use the model to make predictions on new data by passing the data through the model and obtaining the output.

In [ ]:
from langchain.chains import LLMChain    # Import the LLMChain class for building LLM-based workflows
from langchain.llms import GradientLLM   # Import the GradientLLM class for interacting with Gradient AI's API
from langchain.prompts import PromptTemplate # Import the PromptTemplate class for defining how to prompt the LLM
import gradientai
import os # Import the os module for potential file system interactions

In [ ]:
Fine_Tune__adapter_ID = "640e55a6-3046-4824-8831-10e7ae54a1d6_model_adapter"
#Fine_Tune__adapter_ID = Fine_Tune__adapter.id
#  creating a GradientLLM object
llm = GradientLLM(
    model=Fine_Tune__adapter_ID,
    model_kwargs=dict(
        max_generated_token_count=128, # Adjust how your model generates completions
        temperature = 0.7, # randomness
        top_k=50 # Restricts the model to pick from k most likely words,
    ),
)

### **Formatting Prompts**

The model follows the Alpaca prompt format which provides a structured way to input information to the model, guiding it on what kind of output is desired.

In [ ]:
template = """### Instruction: {Instruction} \n\n### Response:"""

prompt = PromptTemplate(template=template, input_variables=["Instruction"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

#### *Example* *inputs* *and* *corresponding* *outputs*

In [ ]:
Question1 = "How does Clostridial Diseases impact milk production, reproductive performance, and overall well-being in cows?"
Question2 = "what are some of the risk factors associated with lameness in dairy cows?"
Question3 = "How dose lameness impact mill production, reproduction and overall well-being in cows"
Question4 = "What diseases are prevelant in dairy small ruminant, and what managment practice can mitigate their impact "
Question5 = "what specific health managment strategies should be implemented to prevent or treat common cow diseases?"


print("Question :\n", Question1)
Answer = llm_chain.run(Instruction=f"{Question1}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question2)
Answer = llm_chain.run(Instruction=f"{Question2}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question3)
Answer = llm_chain.run(Instruction=f"{Question3}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question4)
Answer = llm_chain.run(Instruction=f"{Question4}")
print("Answer :\n", Answer, "\n\n")

print("Question :\n", Question5)
Answer = llm_chain.run(Instruction=f"{Question5}")
print("Answer :\n", Answer, "\n\n")


Question :
 How does Clostridial Diseases impact milk production, reproductive performance, and overall well-being in cows?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer :
 Clostridial Diseases can negatively impact milk production, reproductive performance, and overall well-being in cows. Symptoms can include diarrhea, reduced milk yield, and reproductive issues such as abortion. Early detection and treatment are critical for minimizing the impact on milk production and overall cow health. 


Question :
 what are some of the risk factors associated with lameness in dairy cows?
Answer :
 Risk factors associated with lameness in dairy cows include poor hygiene, inadequate foot care, excessive weight, improper teat positioning, and uneven weight distribution. Management strategies such as routine foot trimming, maintaining proper hygiene, and minimizing stress contribute to reducing the risk of lameness in dairy cows. 


Question :
 How dose lameness impact mill production, reproduction and overall well-being in cows
Answer :
 Lameness in cows can impact mill production by decreasing overall efficiency. Reproduction may be negatively affected as l

## **6.Model Evaluation**

Here we are using two popular automatic evaluation metrics to assess the performance of your LLM:

* BLEU score: This metric calculates the n-gram precision between the generated response and the reference response

* ROUGE score: This metric measures the overlap in word n-grams and longest common subsequences between the generated response and the reference response.

BLEU and ROUGE scores are calculated to compare the generated response with the target response.

In [ ]:
import re
import json
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate
import os

In [ ]:


def compute_rouge_scores(hypotheses, references):
    rouge = Rouge()
    scores = rouge.get_scores(hypotheses, references, avg=True)
    return scores


def compute_bleu_score(target_response, llm_responses):
    bleu_score = corpus_bleu([target_response.split()], [llm_responses.split()])  # Calculate BLEU score
    return bleu_score


def Find_Instruction(input_pattern, input_string):
    matches = re.findall(input_pattern, input_string, re.DOTALL)

    # If there are matches, extract the first one
    extracted_string = None
    if matches:
        extracted_string = matches[0]

    return extracted_string


def Evaluate(Sample=None, count=0):
    print("\n =================================== Evaluation =================================== ")
    input_pattern = r'<s>### Instruction:\n(.*?) \n'
    response_pattern = r'Response:\n(.*?)</s>'
    bleu_scoreS = []
    rouge_scoreS = []

    if count != 0:
        iteration = count - 1
    else:
        iteration = count

    while iteration >= 0:

        input_query = Find_Instruction(input_pattern, Sample[iteration]["inputs"])
        target_response = Find_Instruction(response_pattern, Sample[iteration]["inputs"])

        if input_query and target_response is not None:
            print("\n ---------------------------------------------------------------")
            print("INPUT QUERY:\n", input_query)
            print("\nTARGET RESPONSE:\n", target_response)

            llm_responses = llm_chain.run(Instruction=f"{input_query}")
            print("\nLLM RESPONSE:\n", llm_responses)

            rouge_scores = compute_rouge_scores(llm_responses, target_response)

            bleu_score = compute_bleu_score(target_response, llm_responses)
            print("\nBLEU Score:", bleu_score)
            print("ROUGE Scores:")
            print("\tROUGE-1 F1 Score:", rouge_scores["rouge-1"]["f"])
            print("\tROUGE-2 F1 Score:", rouge_scores["rouge-2"]["f"])
            print("\tROUGE-L F1 Score:", rouge_scores["rouge-l"]["f"])
            rouge_scoreS.append((rouge_scores["rouge-1"]["f"], rouge_scores["rouge-2"]["f"], rouge_scores["rouge-l"]["f"]))
            bleu_scoreS.append(bleu_score)


        iteration -= 1

    if count > 0:
        rouge_scores1 = 0
        rouge_scores2 = 0
        rouge_scores3 = 0
        bleu_scoreA = 0

        for i in bleu_scoreS:
            bleu_scoreA += i
        for i in rouge_scoreS:
            rouge_scores1 += i[0]
            rouge_scores2 += i[1]
            rouge_scores3 += i[2]

        print("\nAverageBLEU Score:", bleu_scoreA)
        print(f"Average ROUGE Scores for {count} samples")
        print("\tAverage ROUGE-1 F1 Score:", rouge_scores1 / count)
        print("\tAverage ROUGE-2 F1 Score:", rouge_scores2 / count)
        print("\tAverageROUGE-L F1 Score:", rouge_scores3 / count)

    print("\n ---------------------------------------------------------------")


Evaluate(Sample=train_dataset, count=3)  # one sample evaluation


 =================================== Evaluation =================================== 

 ---------------------------------------------------------------
INPUT QUERY:
 How does technology contribute to advancements in animal husbandry?

TARGET RESPONSE:
 Technology in animal husbandry includes innovations like automated feeding systems, precision breeding techniques, and health monitoring devices. These advancements enhance efficiency, reduce costs, and improve overall management practices.

LLM RESPONSE:
 Technology contributes to advancements in animal husbandry by enhancing precision in feeding management, optimizing breeding practices, and improving overall efficiency in resource utilization.

BLEU Score: 0
ROUGE Scores:
	ROUGE-1 F1 Score: 0.4166666617447917
	ROUGE-2 F1 Score: 0.08163264811328642
	ROUGE-L F1 Score: 0.37499999507812504

 ---------------------------------------------------------------
INPUT QUERY:
 Elaborate on the challenges faced in modern animal husbandry practices.

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

 ## **7. Intergrating  Retreival-Augmented Generation**

In [ ]:
!pip install gradient_haystack==0.2.0

  Attempting uninstall: gradient_haystack
    Found existing installation: gradient-haystack 0.4.0
    Uninstalling gradient-haystack-0.4.0:
      Successfully uninstalled gradient-haystack-0.4.0


In [ ]:
from gradient_haystack.embedders.gradient_document_embedder import GradientDocumentEmbedder
from gradient_haystack.embedders.gradient_text_embedder import GradientTextEmbedder
from gradient_haystack.generator.base import GradientGenerator
from haystack import Document, Pipeline
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory.document_store import InMemoryDocumentStore
from haystack.components.retrievers.in_memory.embedding_retriever import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.builders.answer_builder import AnswerBuilder
import os
import requests

In [ ]:
os.environ['GRADIENT_ACCESS_TOKEN'] = "t4YxBtnw9hQ4qVB8ia9P4K8XkPLoDoZZ"
os.environ['GRADIENT_WORKSPACE_ID'] = "c571b959-4ce8-474a-b3f3-398c7b347c57_workspace"

fine_tuned_Model_Id = "640e55a6-3046-4824-8831-10e7ae54a1d6_model_adapter"

In [ ]:
document_store = InMemoryDocumentStore()
writer = DocumentWriter(document_store=document_store)


document_embedder = GradientDocumentEmbedder(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
)

# URL of the online repository where the Raw_Text_Data.txt file is located
url = "https://raw.githubusercontent.com/swafey-karanja/Model-training/main/Raw_Text_Data.txt"

# Send a GET request to download the file
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Read the contents of the downloaded file
    text_data = response.text
else:
    # If the request was not successful, print an error message
    print("Failed to download the file from the URL:", url)

docs = [
    Document(content=text_data)
]

indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=document_embedder, name="document_embedder")
indexing_pipeline.add_component(instance=writer, name="writer")
indexing_pipeline.connect("document_embedder", "writer")
indexing_pipeline.run({"document_embedder": {"documents": docs}})

text_embedder = GradientTextEmbedder(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
)

generator = GradientGenerator(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    model_adapter_id=fine_tuned_Model_Id,
    max_generated_token_count=350,
)

100%|██████████| 1/1 [00:00<00:00,  3.03it/s]


In [ ]:
prompt = """You are helpful assistant ment to answer questions relating to animal husbandry. Answer the query, based on the
content in the documents. if you dont know the answer say you don't know.
{{documents}}
Query: {{query}}
\nAnswer:
"""

retriever = InMemoryEmbeddingRetriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt)

rag_pipeline = Pipeline()
rag_pipeline.add_component(instance=text_embedder, name="text_embedder")
rag_pipeline.add_component(instance=retriever, name="retriever")
rag_pipeline.add_component(instance=prompt_builder, name="prompt_builder")
rag_pipeline.add_component(instance=generator, name="generator")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")
rag_pipeline.connect("generator.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")
rag_pipeline.connect("text_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")


def LLM_Run(question):
    result = rag_pipeline.run(
        {
            "text_embedder": {"text": question},
            "prompt_builder": {"query": question},
            "answer_builder": {"query": question}
        }
    )
    return result["answer_builder"]["answers"][0].data

In [34]:
Query = "what is the best way of preventing bacterial infections?"
print(LLM_Run(Query))

Preventing bacterial infections involves maintaining proper hygiene, minimizing stress factors, and implementing vaccination programs. Regular health checks, prompt treatment of any signs of infection, and maintaining optimal environmental conditions contribute to reducing the risk of bacterial infections in animal husbandry.
